In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
rc('font', **{ "size":15,}) #**{,,
rc('text', usetex=True)
import numpy.random as rnd
rnd.seed()
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import scipy.linalg as la
import scipy.sparse.csgraph as graph
import pickle as cp
from scipy.special import gamma
from scipy.optimize import curve_fit as fit
from scipy.optimize import brentq
import scipy.fft as fft
from scipy.integrate import solve_ivp as ode
l10 = lambda x: np.log(x) / np.log(10)
import csv
from datetime import date


In [ ]:
date = []
fips = []
case = []
with open("us-counties-2020.csv") as f:
    first=1
    for line in f:
        if first: 
            first = 0
            continue
        
        spl = line.split(",")
        if len(spl[3]) != 5:
            continue
        date.append(spl[0])
        fips.append(spl[3])
        #
        #    print(line)
           # raise ValueError
        case.append(spl[4])

In [ ]:
database = dict()
for day, code, c in zip(date, fips, case):
    if not day in database: 
        database[day] = dict()
    database[day][code] = float(c)

In [ ]:
geopoints = []
fips1 = []
with open("us-county-boundaries.csv") as f:
    first=1
    for line in f:
        if first: 
            first = 0
            continue
        spl = line.split(";")
        geopoints.append(spl[0])
        fips1.append(spl[2] + spl[3])
xy = np.array([ [float(x) for x in l.split(",")][::-1] for l in geopoints])
carte = {code:coord for code, coord in zip(fips1,xy)}

In [ ]:
plt.scatter(xy[:, 0], xy[:, 1],   s=1)
plt.xlim(-130, -60)
plt.ylim(25, 50)

In [ ]:
for jour in ["2020-02-28", "2020-03-01","2020-03-05"]: 
#plt.scatter(xy[:, 0], xy[:, 1], s=1, alpha=.2, c="k")
    dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
    #plt.scatter(dataofday[:,0], dataofday[:,1], c = dataofday[:,2], s = 2, cmap = "RdBu")
    #plt.colorbar()
    #plt.xlim(-130, -60)
    #plt.ylim(25, 50)
    
    N = len(dataofday)
    print(jour, N)
       
    ths = dataofday[:, 0] / 180 * np.pi
    phis= dataofday[:, 1] / 180 * np.pi
    xs = np.cos(ths) * np.cos(phis)
    ys = np.sin(ths) * np.cos(phis)
    zs = np.sin(phis)
    M = np.arccos( np.outer(xs, xs) + np.outer(ys, ys) + np.outer(zs, zs) ) * 6300  
   # M = [[dis(i,j) for i in range(N)] for j in range(N)]
    bs = np.exp(np.linspace(np.log(20), np.log(10000), 20))
    Ncbs = []
    for b in bs:
        print(".", end="")
        T = M < b
        Ncb, nouse = graph.connected_components(T)
        Ncbs.append(Ncb)
    plt.loglog(bs, np.array(Ncbs) , ".-", label=jour)
plt.legend()
plt.xlabel("$b$ (km)")
plt.ylabel("$N_c(b)$")

In [ ]:
plt.scatter(dataofday[:,0], dataofday[:,1], c = dataofday[:,2], s = 2, cmap = "RdBu")
plt.colorbar()
plt.xlim(-130, -60)
plt.ylim(25, 50)

In [ ]:
for jour in ["2020-03-01","2020-03-03",  "2020-03-05", "2020-03-07", ]: 

    dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
    #plt.scatter(dataofday[:,0], dataofday[:,1], c = dataofday[:,2], s = 2, cmap = "RdBu")
    #plt.colorbar()
    #plt.xlim(-130, -60)
    #plt.ylim(25, 50)
    print(sum([x for u,v,x in dataofday]))

In [ ]:
for jour in ["2020-03-01","2020-03-03",  "2020-03-05", ]: 
#plt.scatter(xy[:, 0], xy[:, 1], s=1, alpha=.2, c="k")
    dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
    #plt.scatter(dataofday[:,0], dataofday[:,1], c = dataofday[:,2], s = 2, cmap = "RdBu")
    #plt.colorbar()
    #plt.xlim(-130, -60)
    #plt.ylim(25, 50)
    
    N = len(dataofday)
    print(jour, N)
       
    ths = dataofday[:, 0] / 180 * np.pi
    phis= dataofday[:, 1] / 180 * np.pi
    xs = np.cos(ths) * np.cos(phis)
    ys = np.sin(ths) * np.cos(phis)
    zs = np.sin(phis)
    M = np.arccos( np.outer(xs, xs) + np.outer(ys, ys) + np.outer(zs, zs) ) * 6300  
   # M = [[dis(i,j) for i in range(N)] for j in range(N)]
    bs = np.exp(np.linspace(np.log(20), np.log(5000), 15))
    Ncbs = []
    for b in bs:
        print(".", end="")
        T = M < b
        Ncb, nouse = graph.connected_components(T)
        Ncbs.append(Ncb)
    plt.loglog(bs, np.array(Ncbs)  , ".-", label=jour)

plt.xlabel("$b$ (km)")
plt.ylabel("$N_c(b)/ N_c(b)$")
alpha, d = .6, 2
plt.plot(bs, bs ** - (alpha * d / (alpha + d)) * 100, "k--", lw=2, label=r"$\alpha=0.6$")
plt.legend()

In [ ]:
fig, ax = plt.subplots(2,3, figsize = (12,6),sharex="all", sharey="all")
ax = ax.reshape(-1)

for k,jour in enumerate(["2020-03-01","2020-03-03", "2020-03-05", "2020-03-07", "2020-03-20", "2020-03-30"]): 
    plt.sca(ax[k])
    plt.scatter(-xy[:, 0], xy[:, 1], s=1, alpha=.1, c="k",rasterized=1)
    dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
    plt.scatter(-dataofday[:,0], dataofday[:,1], c = (dataofday[:,2] > 0), s = 2, rasterized=1)
   # plt.colorbar()
    plt.xlim(130, 60)
    plt.ylim(25, 50)
    if k // 3 > 0: plt.xlabel("longitude (deg, W)")
    
    if k % 3==0: plt.ylabel("latitude (deg, N)")
    plt.title(jour)
plt.tight_layout()
#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/US.pdf")

In [ ]:
d = 2
alpha=.6
fig, ax = plt.subplots(1,2, figsize = (12, 4))
ax = ax.reshape(-1)
for jour in ["2020-03-01", "2020-03-03", "2020-03-05", "2020-03-07", "2020-03-20", "2020-03-30" ]: 
#plt.scatter(xy[:, 0], xy[:, 1], s=1, alpha=.2, c="k")
    dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
    #plt.scatter(dataofday[:,0], dataofday[:,1], c = dataofday[:,2], s = 2, cmap = "RdBu")
    #plt.colorbar()
    #plt.xlim(-130, -60)
    #plt.ylim(25, 50)
    S = (sum([x for u,v,x in dataofday]))
    #N = len(dataofday)
    #print(jour, N)
       
    ths = dataofday[:, 0] / 180 * np.pi
    phis= dataofday[:, 1] / 180 * np.pi
    xs = np.cos(ths) * np.cos(phis)
    ys = np.sin(ths) * np.cos(phis)
    zs = np.sin(phis)
    M = np.arccos( np.outer(xs, xs) + np.outer(ys, ys) + np.outer(zs, zs) ) * 6300  
   # M = [[dis(i,j) for i in range(N)] for j in range(N)]
    bs = np.exp(np.linspace(np.log(20), np.log(5000), 20))
    Ncbs = []
    for b in bs:
        print(".", end="")
        T = M < b
        Ncb, nouse = graph.connected_components(T)
        Ncbs.append(Ncb)
    ax[0].loglog(bs, np.array(Ncbs) , ".-", label=jour)
    ax[1].loglog(bs, np.array(Ncbs) / S ** (d / (d + alpha)), ".-", label=jour)
ax[1].plot(bs, (bs/ 50) ** - (alpha * d / (alpha + d)) , "k--", lw=2, label=r"$\alpha=0.6$")
plt.sca(ax[0])
#plt.legend(bbox_to_anchor = (1,1))
plt.xlabel(r"$b$ (km)")
plt.ylabel(r"$N_c(b)$")
plt.sca(ax[1])
plt.legend(bbox_to_anchor = (1,1))
plt.xlabel(r"$b$ (km)")
plt.ylabel(r"$N_c(b) / S^{{d}/(\alpha + d)}$")
plt.tight_layout()
plt.ylim(1e-2, 2)
#plt.savefig("../Dropbox/Apps/Overleaf/galton-watson/paper1/US2.pdf")

In [ ]:

jour =  "2020-03-30"

plt.scatter(-xy[:, 0], xy[:, 1], s=1, alpha=.1, c="k")
dataofday = np.array([ [carte[code][0],carte[code][1], count] for code, count in database[jour].items()])
plt.scatter(-dataofday[:,0], dataofday[:,1], c = (dataofday[:,2] > 0), s = 2 )
# plt.colorbar()
plt.xlim(130, 60)
plt.ylim(25, 50)
if k in [2,3]: plt.xlabel("longitude (deg, W)")

if k in [0,2]: plt.ylabel("latitude (deg, N)")
plt.title(jour)
plt.tight_layout()